# Multi-Species Multi-Reaction model

## Model Equations

Here we briefly outline the models used for the open-circuit potential and solid phase transport used in the MSMR model, as described in Baker and Verbrugge (2018). The remaining physics is modelled differently depending on which options are selected. By default, the rest of the battery model is as described in Maquis et al. (2019).

## Thermodynamics
The MSMR model is developed by assuming that all electrochemical reactions at the electrode/electrolyte interface in a lithium insertion cell can be expressed in the form 
$$ \text{Li}^{+} + \text{e}^{-} + \text{H}_{j} \rightleftharpoons  (\text{Li--H})_{j}.$$
For each species $j$, a vacant host site $\text{H}_{j}$ can accommodate one lithium leading to a filled host site $(\text{Li--H})_{j}$. The OCV for this reaction is written as
$$ U_j = U_j^0 + \frac{\omega_j}{f}\log\left(\frac{X_j - x_j}{x_j}\right),$$
where $f = (RT)/F$, and $R$, $T$, and $F$ are the universal gas constant, temperature in Kelvin, and Faraday’s constant, respectively. Here $X_j$ represents the total fraction of available host sites which can be occupied by species $j$, $x_j$ is the fraction of filled sites occupied by species $j$, $U_j^0$ is a concentration independent standard electrode potential, and the $\omega_j$ is an unitless parameter that describes the level of disorder of the reaction represented by gallery $j$. 

The equation for each reaction can be inverted to give 
$$x_j = \frac{X_j}{1+\exp[f(U-U_j^0)/\omega_j]}.$$
The overall electrode state of charge is given by summing the fractional occupancies 
$$x = \sum_j x_j = \sum_j \frac{X_j}{1+\exp[f(U-U_j^0)/\omega_j]},$$
which is an explicit closed form expression for the inverse of the OCV. This is opposite to many battery models where one typically gives the OCV as an explicit function of the state of charge (or stoichiometry).

At a particle interface with the electrolyte, local equilibrium requires that 
$$U_j = U(x) \quad \forall j.$$

## Solid phase transport
Within the MSMR framework, the flux within the particles is expressed in terms of gradient of the chemical potential
$$N = -c_{\text{T}}x\frac{D}{RT}\nabla \mu + x(N+N_{\text{H}}),$$
where $N$ is the flux of lithiated sites, $N_{\text{H}}$ is the flux of unlithiated sites, $c_{\text{T}}$ is the total concentration of lithiated and delithiated sites, and $D$ is a diffusion coefficient. Ignoring volumetric expansion during lithiation, the total flux of sites vanishes
$$N+N_{\text{H}}.$$ 
It can then be shown that 
$$N = c_{\text{T}}fDx(1-x)\frac{\text{d}U}{\text{d}x}\nabla x.$$

A mass balance in the solid phase then gives
$$\frac{\partial x}{\partial t} = -\nabla\cdot\left(x(1-x)fD\frac{\text{d}U}{\text{d}x}\nabla x\right),$$
which, for a radially symmetric spherical particle, must be solved subject to the boundary conditions
$$N\big\vert_{r=0} = 0, \quad N\big\vert_{r=R} = \frac{j}{F},$$
where $j$ is the interfacial current density and $R$ is the particle radius. This must be supplemented with a suitable initial condition for the electrode state of charge.

Solution of this problem requires evaluate of the function $U(x)$ and the derivative $\text{d}U/\text{d}x$, but these functions cannot be explicitly integrated. This problem can be avoided by replacing the dependent variable $x$ with a new dependent variable $U$ subject to the transformation 
$$x = \sum_j \frac{X_j}{1+\exp[f(U-U_j^0)/\omega_j]}.$$
This gives the following equation for mass balance within the particles
$$\frac{\text{d}U}{\text{d}x}\frac{\partial U}{\partial t} = -\nabla\cdot\left(x(1-x)fD\nabla x\right),$$

which must be solved along with the transformed boundary and initial conditions.

## Example solving MSMR using PyBaMM
Below we show how to set up and solve a CCCV experiment using the MSMR model in PyBaMM. We use an example parameter set based on an Gr vs NMC cell similar to the LG M50.

We begin by importing pybamm and numpy

In [20]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np

Next we load in the model. We choose to use the DFN along with our MSMR model for the open-circuit potential and solid phase (particle) transport

In [21]:
model = pybamm.lithium_ion.DFN(
    {
        "open-circuit potential": "MSMR",
        "particle": "MSMR",
    }
)

parameter_values = pybamm.ParameterValues("MSMR_Example")

We can also add variables for the individual electrode reaction as described in the MSMR model. We cannot create these variables until _after_ we have chosen some parameter values since we do not know in advance how many reactions have been used to describe thermodynamics of each electrode. The number of reactions is selected as part of parameterizing a particular material.

In [22]:
model.set_msmr_reaction_variables(parameter_values)

The individual reactions are given variables names `xj_k` where `k` can be `n` or `p` to denote the negative or positive electrode, and `j` is the reaction index. E.g. the variable for the second reaction in the negative electrode can be accessed as

In [23]:
xn_2 = model.variables["x2_n"]

Next we define our experiment, before creating and solving a simulation

In [24]:
experiment = pybamm.Experiment(
    [
        (
            "Discharge at 1C for 1 hour or until 3 V",
            "Rest for 1 hour",
            "Charge at C/3 until 4 V",
            "Hold at 4 V until 10 mA",
            "Rest for 1 hour",
        ),
    ]
)
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment)
sim.solve()

Finally we can plot the results. In the MSMR model we can look at both the potential and stoichiometry as a function of position through the electrode and within the particle

In [25]:
sim.plot(
    [
        "Negative particle stoichiometry",
        "Positive particle stoichiometry",
        "X-averaged negative electrode open-circuit potential [V]",
        "X-averaged positive electrode open-circuit potential [V]",        
        "Negative particle potential [V]",
        "Positive particle potential [V]",
        "Current [A]",
        "Voltage [V]",
    ],
    variable_limits="tight",  # make axes tight to plot at each timestep
)

interactive(children=(FloatSlider(value=0.0, description='t', max=6.2959964898302045, step=0.06295996489830205…

We can also look at the individual reactions

In [26]:
sim.plot(
    [
        [f"X-averaged x{i}_n" for i in range(6)],
        [f"X-averaged x{i}_p" for i in range(4)],
        "Current [A]",
        "X-averaged negative electrode open-circuit potential [V]",
        "X-averaged positive electrode open-circuit potential [V]",
        "Voltage [V]",
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=6.2959964898302045, step=0.06295996489830205…

## References

The relevant papers for this notebook are:

In [27]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Daniel R Baker and Mark W Verbrugge. Multi-species, multi-reaction model for porous intercalation electrodes: part i. model formulation and a perturbation solution for low-scan-rate, linear-sweep voltammetry of a spinel lithium manganese oxide electrode. Journal of The Electrochemical Society, 165(16):A3952, 2018.
[3] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[4] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array program